In [1]:
# load required libraries
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
%matplotlib inline 

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)

import lightgbm as lgb
from sklearn import feature_extraction
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from tqdm import tqdm, tqdm_notebook

tqdm.monitor_interval = 0

from itertools import product
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', module='scipy', message='^internal gelsd')

### Load the data

Let's load the data from the hard drive first.

In [2]:
sales = pd.read_csv('../datasets/salesdata/sales_train.csv.gz')
shops = pd.read_csv('../datasets/salesdata/shops.csv')
items = pd.read_csv('../datasets/salesdata/items.csv')
item_cats = pd.read_csv('../datasets/salesdata/item_categories.csv')
test = pd.read_csv('../datasets/salesdata/test.csv.gz')

In [3]:
test_block = sales['date_block_num'].iloc[-1] + 1
test['date_block_num'] = test_block
test.head()

,ID,shop_id,item_id,date_block_num
0,0,5,5037,34
1,1,5,5320,34
2,2,5,5233,34
3,3,5,5232,34
4,4,5,5268,34


In [4]:
target_range = [0, 20]

## Panda's basics

### Question 1

What was the maximum total revenue among all the shops in September, 2014?

Hereinafter revenue refers to total sales minus value of goods returned.
Hints:

- Sometimes items are returned, find such examples in the dataset.
- It is handy to split date field into [day, month, year] components and use df.year == 14 and df.month == 9 in order to select target subset of dates.
- You may work with date feature as with srings, or you may first convert it to pd.datetime type with pd.to_datetime function, but do not forget to set correct format argument.

In [7]:
trans = sales[sales['date'].str.endswith('09.2014')]
max_revenue = (trans['item_price'] * trans['item_cnt_day']).groupby(trans['shop_id']).sum().max()
print(max_revenue)

7982852.199999956


Q2. What item category generated the highest revenue in summer 2014?

Submit id of the category found.

Here we call "summer" the period from June to August.

*Hints:*

Note, that for an object x of type pd.Series: x.argmax() returns index of the maximum element. pd.Series can have non-trivial index (not [1, 2, 3, ... ]).

In [8]:
trans = sales[sales['date'].str.match('.*0[6-8].2014$')]
trans = trans[trans['item_cnt_day'] > 0]
trans = pd.merge(trans, items, how='left', on='item_id')
category_id_with_max_revenue = (trans['item_price'] * trans['item_cnt_day']).groupby(trans['item_category_id']).sum().argmax()
print(category_id_with_max_revenue)

20


Q3. How many items are there, such that their price stays constant (to the best of our knowledge) during the whole period of time?
Let's assume, that the items are returned for the same price as they had been sold.

In [9]:
num_items_constant_price = (sales['item_price'].groupby(sales['item_id']).nunique() == 1).sum()
print(num_items_constant_price)

5926


Q4. What was the variance of the number of sold items per day sequence for the shop with `shop_id = 25` in December, 2014? 

Do not count the items, that were sold but returned back later.

Fill total_num_items_sold and days arrays, and plot the sequence with the code below.

Then compute variance. Remember, there can be differences in how you normalize variance (biased or unbiased estimate, see link). 

Compute unbiased estimate (use the right value for ddof argument in pd.var or np.var).

In [10]:
shop_id = 25

trans = sales[sales['shop_id'] == shop_id]
trans = trans[trans['date'].str.endswith('12.2014')]
trans = trans['item_cnt_day'].groupby(trans['date']).sum()
total_num_items_sold = trans.values
days = trans.index.to_datetime()

# Plot it
plt.plot(days, total_num_items_sold)
plt.ylabel('Num items')
plt.xlabel('Day')
plt.title("Daily revenue for shop_id = 25")
plt.show()

total_num_items_sold_var = trans.var()
grader.submit_tag('total_num_items_sold_var', total_num_items_sold_var)

AttributeError: 'Index' object has no attribute 'to_datetime'